## 2.3. ER Schema Creation - Feodo Tracker

## Feodo Tracker Entities 

- Entries
- Tags
- Malwares
- Alias


In [1]:
import datetime
import os
from OTXv2 import OTXv2
import time
import json 
import pandas as pd 
import requests
import urllib3
import json
import sys
import numpy as np

In [2]:
pd.options.display.max_colwidth = 500

In [3]:
file_FT = r'C:\Users\leona\OneDrive\Desktop\Tesi\Pipeline\Datasets\Pipeline\IocSegnalations\PostDataCollection\Feodo_Tracker.csv'

# THREAT FOX

In [4]:
df = pd.read_csv(file_FT, low_memory = False )
df = df.drop('Unnamed: 0', axis=1)
df.head(30)

,ip_address,port,status,hostname,as_number,as_name,country,first_seen,last_online,malware
0,89.101.97.139,443,offline,089-101-097139.ntlworld.ie,6830,"LIBERTYGLOBAL Liberty Global formerly UPC Broadband Holding, aka AORTA",IE,2021-09-29 08:42:47,2023-08-25,QakBot
1,41.228.22.180,443,offline,NaN,37693,TUNISIANA,TN,2021-09-29 08:42:51,2023-08-25,QakBot
2,144.139.47.206,443,offline,NaN,1221,ASN-TELSTRA Telstra Corporation Ltd,AU,2021-09-30 21:25:38,2023-08-25,QakBot
3,41.86.42.158,995,offline,NaN,36958,CWSeychelles-AS,SC,2021-10-08 14:48:30,2023-08-25,QakBot
4,63.143.92.99,995,offline,NaN,33576,DIG001,JM,2021-10-08 14:48:31,2023-08-25,QakBot
5,24.178.196.158,2222,offline,024-178-196-158.biz.spectrum.com,20115,CHARTER-20115,US,2021-12-17 22:33:06,2023-08-23,QakBot
6,196.203.37.215,80,offline,NaN,37719,Hexabyte-AS,TN,2022-02-23 07:25:23,2023-08-25,QakBot
7,70.46.220.114,443,offline,70.46.220.114.nw.nuvox.net,7029,WINDSTREAM,US,2022-03-03 18:06:08,2023-08-25,QakBot
8,148.64.96.100,443,offline,148-64-96-100.PUBLIC.monkeybrains.net,32329,MONKEYBRAINS,US,2022-03-15 13:23:59,2023-08-25,QakBot
9,172.115.177.204,2222,offline,cpe-172-115-177-204.socal.res.rr.com,20001,TWC-20001-PACWEST,US,2022-03-16 19:53:57,2023-08-25,QakBot


In [5]:
df.columns

Index(['ip_address', 'port', 'status', 'hostname', 'as_number', 'as_name',
       'country', 'first_seen', 'last_online', 'malware'],
      dtype='object')

## Merge columns
Merge column ip-address and port into a single column


In [6]:
df['ip'] = df['ip_address'].astype(str).str.cat(df['port'].astype(str), sep=':')
df = df.drop(['ip_address'], axis=1)
df = df.drop(['port'], axis=1)
df

,status,hostname,as_number,as_name,country,first_seen,last_online,malware,ip
0,offline,089-101-097139.ntlworld.ie,6830,"LIBERTYGLOBAL Liberty Global formerly UPC Broadband Holding, aka AORTA",IE,2021-09-29 08:42:47,2023-08-25,QakBot,89.101.97.139:443
1,offline,NaN,37693,TUNISIANA,TN,2021-09-29 08:42:51,2023-08-25,QakBot,41.228.22.180:443
2,offline,NaN,1221,ASN-TELSTRA Telstra Corporation Ltd,AU,2021-09-30 21:25:38,2023-08-25,QakBot,144.139.47.206:443
3,offline,NaN,36958,CWSeychelles-AS,SC,2021-10-08 14:48:30,2023-08-25,QakBot,41.86.42.158:995
4,offline,NaN,33576,DIG001,JM,2021-10-08 14:48:31,2023-08-25,QakBot,63.143.92.99:995
...,...,...,...,...,...,...,...,...,...
333,offline,NaN,36947,ALGTEL-AS,DZ,2023-08-24 23:29:43,2023-08-25,QakBot,41.97.47.7:443
334,offline,host-92-9-44-234.as13285.net,13285,OPALTELECOM-AS TalkTalk Communications Limited,GB,2023-08-25 01:29:36,2023-08-25,QakBot,92.9.44.234:2222
335,offline,NaN,15502,VODAFONE-IRELAND-ASN,IE,2023-08-25 10:08:52,2023-08-25,QakBot,78.152.198.132:443
336,offline,NaN,45528,TIKONAIN-AS Tikona Infinet Ltd.,IN,2023-08-25 11:24:40,2023-08-25,QakBot,113.193.95.229:443


### Column Filtering

In [7]:
df = df.drop([ 'hostname', 'as_number', 'as_name', 'country'], axis=1)
df

,status,first_seen,last_online,malware,ip
0,offline,2021-09-29 08:42:47,2023-08-25,QakBot,89.101.97.139:443
1,offline,2021-09-29 08:42:51,2023-08-25,QakBot,41.228.22.180:443
2,offline,2021-09-30 21:25:38,2023-08-25,QakBot,144.139.47.206:443
3,offline,2021-10-08 14:48:30,2023-08-25,QakBot,41.86.42.158:995
4,offline,2021-10-08 14:48:31,2023-08-25,QakBot,63.143.92.99:995
...,...,...,...,...,...
333,offline,2023-08-24 23:29:43,2023-08-25,QakBot,41.97.47.7:443
334,offline,2023-08-25 01:29:36,2023-08-25,QakBot,92.9.44.234:2222
335,offline,2023-08-25 10:08:52,2023-08-25,QakBot,78.152.198.132:443
336,offline,2023-08-25 11:24:40,2023-08-25,QakBot,113.193.95.229:443


### Column Renaming

In [8]:
Feodo_Tracker_ENTRIES_df = df.rename(columns={ "last_online": "last_seen", "ip" : "ioc" })

Feodo_Tracker_ENTRIES_df["ioc_type"] = "ip:port"
Feodo_Tracker_ENTRIES_df["source"] = "Feodo Tracker"

Feodo_Tracker_ENTRIES_df['threat_type'] = 'botnet'
Feodo_Tracker_ENTRIES_df['reference'] = 'https://feodotracker.abuse.ch/'
Feodo_Tracker_ENTRIES_df['reporter'] = 'Feodo Tracker blocklist'

Feodo_Tracker_ENTRIES_df['ID_ENTRY'] = Feodo_Tracker_ENTRIES_df.index
Feodo_Tracker_ENTRIES_df

,status,first_seen,last_seen,malware,ioc,ioc_type,source,threat_type,reference,reporter,ID_ENTRY
0,offline,2021-09-29 08:42:47,2023-08-25,QakBot,89.101.97.139:443,ip:port,Feodo Tracker,botnet,https://feodotracker.abuse.ch/,Feodo Tracker blocklist,0
1,offline,2021-09-29 08:42:51,2023-08-25,QakBot,41.228.22.180:443,ip:port,Feodo Tracker,botnet,https://feodotracker.abuse.ch/,Feodo Tracker blocklist,1
2,offline,2021-09-30 21:25:38,2023-08-25,QakBot,144.139.47.206:443,ip:port,Feodo Tracker,botnet,https://feodotracker.abuse.ch/,Feodo Tracker blocklist,2
3,offline,2021-10-08 14:48:30,2023-08-25,QakBot,41.86.42.158:995,ip:port,Feodo Tracker,botnet,https://feodotracker.abuse.ch/,Feodo Tracker blocklist,3
4,offline,2021-10-08 14:48:31,2023-08-25,QakBot,63.143.92.99:995,ip:port,Feodo Tracker,botnet,https://feodotracker.abuse.ch/,Feodo Tracker blocklist,4
...,...,...,...,...,...,...,...,...,...,...,...
333,offline,2023-08-24 23:29:43,2023-08-25,QakBot,41.97.47.7:443,ip:port,Feodo Tracker,botnet,https://feodotracker.abuse.ch/,Feodo Tracker blocklist,333
334,offline,2023-08-25 01:29:36,2023-08-25,QakBot,92.9.44.234:2222,ip:port,Feodo Tracker,botnet,https://feodotracker.abuse.ch/,Feodo Tracker blocklist,334
335,offline,2023-08-25 10:08:52,2023-08-25,QakBot,78.152.198.132:443,ip:port,Feodo Tracker,botnet,https://feodotracker.abuse.ch/,Feodo Tracker blocklist,335
336,offline,2023-08-25 11:24:40,2023-08-25,QakBot,113.193.95.229:443,ip:port,Feodo Tracker,botnet,https://feodotracker.abuse.ch/,Feodo Tracker blocklist,336


### MALWARES - Feodo Tracker

- MALWARES ( MALWARE_NAME )


In [9]:
Feodo_Tracker_MALWARES_df = pd.DataFrame(df['malware']).drop_duplicates()
        
Feodo_Tracker_MALWARES_df = Feodo_Tracker_MALWARES_df.reset_index(drop = True)

Feodo_Tracker_MALWARES_df['type'] = 'unknown'
Feodo_Tracker_MALWARES_df

,malware,type
0,QakBot,unknown
1,Pikabot,unknown


### ENTRIES - Threat Fox

- ENTRIES ( // ID_ENTRY, SOURCE // , ioc, ioc_type, threat_type, malware, first_seen, last_seen, reporter, reference )

In [10]:
columns_ordered = ['ID_ENTRY', 'source', 'ioc', 'ioc_type', 'threat_type', 'malware', 'first_seen', 'last_seen', 'reporter', 'reference']

In [11]:
Feodo_Tracker_ENTRIES_df = Feodo_Tracker_ENTRIES_df[columns_ordered]

Feodo_Tracker_ENTRIES_df

,ID_ENTRY,source,ioc,ioc_type,threat_type,malware,first_seen,last_seen,reporter,reference
0,0,Feodo Tracker,89.101.97.139:443,ip:port,botnet,QakBot,2021-09-29 08:42:47,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
1,1,Feodo Tracker,41.228.22.180:443,ip:port,botnet,QakBot,2021-09-29 08:42:51,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
2,2,Feodo Tracker,144.139.47.206:443,ip:port,botnet,QakBot,2021-09-30 21:25:38,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
3,3,Feodo Tracker,41.86.42.158:995,ip:port,botnet,QakBot,2021-10-08 14:48:30,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
4,4,Feodo Tracker,63.143.92.99:995,ip:port,botnet,QakBot,2021-10-08 14:48:31,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
...,...,...,...,...,...,...,...,...,...,...
333,333,Feodo Tracker,41.97.47.7:443,ip:port,botnet,QakBot,2023-08-24 23:29:43,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
334,334,Feodo Tracker,92.9.44.234:2222,ip:port,botnet,QakBot,2023-08-25 01:29:36,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
335,335,Feodo Tracker,78.152.198.132:443,ip:port,botnet,QakBot,2023-08-25 10:08:52,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/
336,336,Feodo Tracker,113.193.95.229:443,ip:port,botnet,QakBot,2023-08-25 11:24:40,2023-08-25,Feodo Tracker blocklist,https://feodotracker.abuse.ch/


# SAVE into ..\PostDataStructurization\Threat_Fox\ folder

In [12]:
dataframes_dict = {}

# Add the DataFrames to the dictionary with keys
dataframes_dict['Entries']  = Feodo_Tracker_ENTRIES_df
dataframes_dict['Malwares'] = Feodo_Tracker_MALWARES_df

In [13]:
save_directory = "C:\\Users\\leona\\OneDrive\\Desktop\\Tesi\\Pipeline\\Datasets\\Pipeline\\IocSegnalations\\PostDataStructurization\\Feodo_Tracker\\"
for key, df in dataframes_dict.items():
    file_name = f"{key}.csv"
    file_path = save_directory + file_name
    df.to_csv(file_path, index=False)